---
# Classificação de Demandas do Fala.BR
---
## Projeto do Bootcamp Machine Learning (Anderson Monteiro e Léo Maranhão de Mello)


### Problema
O problema consiste em classificar as demandas recebidas pela Ouvidoria da SUSEP, por meio do sistema Fala.BR.

### Solução de IA
Para fins de comparação com a solução em ML, utilizaremos uma solução em LLM, além de alguns testes sobre aumento de dados (data aumengtation). A LLM escolhida foi a DistilBERT, que necessariamente não desempenha melhor que a google-bert/bert-base-uncased, mas consegue entregar ótimos resultados com otimização de 60% na execução.

In [1]:
# general imports
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

# others
import torch
from google.colab import drive

### Carga dos Dados

In [2]:
# Monta a pasta da nuvem
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# Conecta a base de dados
df = pd.read_csv("/content/drive/MyDrive/data/dados_integralizados.csv",sep=';', index_col=False)
display(df.head())
df.shape

,Demanda,Categoria
0,"Documentos encaminhados no dia 33/33/3333, e a...",Cadastro
1,"Prezados,Tenho dúvidas relacionadas à Circular...",Seguro de Automoveis
2,"Sou segurada, possuo seguro junto a HDI SEGURO...",Seguro de Automoveis
3,"Boa tarde, Recebo um pequeno seguro-previdenci...",Seguro de Pessoas
4,Minha Susep de nº 333333333 está suspensa e nã...,Cadastro


(3254, 2)

In [4]:
# Transforma os valores de target de texto para numero
le = LabelEncoder()
df['Categoria'] = le.fit_transform(df['Categoria'])

In [5]:
# Separa em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df['Demanda'], df['Categoria'], test_size=0.2, random_state=1)

In [6]:
# Cria os tokens
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
# Separa os arquivos de texto em treino e teste
train_texts = X_train.tolist()
test_texts = X_test.tolist()

In [8]:
# Transforma os valores de texto em  sequencias de numeros
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [9]:

class TicketDataset(torch.utils.data.Dataset):
# Classe básica para pegar o par texto x target e mandar para o pytorch
# Pytorch é necessário aqui como um ferramental para pegar os tokens e treinar
# no LLM
    def __init__(self, encodings, labels): # inicializa os parametros
        self.encodings = encodings
        self.labels = labels
    def __len__(self): # retorna quantidade de labels quando chamada por len()
        return len(self.labels)
    def __getitem__(self, idx): # retorna um item quando chamada por []
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [10]:
# Cria os datasets
train_dataset = TicketDataset(train_encodings, y_train.tolist())
test_dataset = TicketDataset(test_encodings, y_test.tolist())

In [11]:
# essa bosta eh um dict contendo os tensors e os labels
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(le.classes_))

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# configuracoes de treinamento
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # aumenta a taxa de aprendizado a cada step
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# simplifica o loop de treinamento (aquela parte do loop)
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.614700,1.465191
2,0.758000,0.666157
3,0.510000,0.518853


TrainOutput(global_step=489, training_loss=1.1996464349009508, metrics={'train_runtime': 407.3322, 'train_samples_per_second': 19.171, 'train_steps_per_second': 1.2, 'total_flos': 1034567050226688.0, 'train_loss': 1.1996464349009508, 'epoch': 3.0})

In [14]:
preds_output = trainer.predict(test_dataset)

In [16]:
preds = torch.argmax(torch.tensor(preds_output.predictions), dim=1)

y_pred = preds.numpy()
y_true = y_test.to_numpy()

print("Relatorio de Classificacao:\n", classification_report(y_true, y_pred, target_names=le.classes_))
print("Acuracia:", accuracy_score(y_true, y_pred))

Classification Report:
                                  precision    recall  f1-score   support

                       Cadastro       0.90      0.72      0.80        87
                    DPVAT/SPVAT       1.00      0.91      0.95        81
               Não identificada       0.80      0.64      0.71        75
                         Outros       0.76      0.77      0.77        71
Previdência Complementar Aberta       1.00      0.97      0.98        67
                Seguro Garantia       1.00      0.92      0.96        61
           Seguro de Automoveis       0.74      0.80      0.77        44
                Seguro de Danos       0.98      0.88      0.93        72
              Seguro de Pessoas       0.62      0.95      0.75        93

                       accuracy                           0.84       651
                      macro avg       0.87      0.84      0.85       651
                   weighted avg       0.86      0.84      0.84       651

Accuracy: 0.840245775729

In [17]:
le.classes_


array(['Cadastro', 'DPVAT/SPVAT', 'Não identificada', 'Outros',
       'Previdência Complementar Aberta', 'Seguro Garantia',
       'Seguro de Automoveis', 'Seguro de Danos', 'Seguro de Pessoas'],
      dtype=object)